In [419]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix


# Reading data

In [420]:
data1 = pd.read_csv('../DATA/Dataset1.csv')
data1


,N,P,K,pH,EC,OC,S,Zn,Fe,Cu,Mn,B,OM,Fertility
0,138,8.6,560,7.46,0.62,0.70,5.90,0.24,0.31,0.77,8.71,0.11,1.2040,0
1,213,7.5,338,7.62,0.75,1.06,25.40,0.30,0.86,1.54,2.89,2.29,1.8232,0
2,163,9.6,718,7.59,0.51,1.11,14.30,0.30,0.86,1.57,2.70,2.03,1.9092,0
3,157,6.8,475,7.64,0.58,0.94,26.00,0.34,0.54,1.53,2.65,1.82,1.6168,0
4,270,9.9,444,7.63,0.40,0.86,11.80,0.25,0.76,1.69,2.43,2.26,1.4792,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,276,9.2,370,7.62,0.62,0.49,6.64,0.42,3.57,0.63,6.48,0.32,0.8428,1
881,320,13.8,391,7.38,0.65,1.07,5.43,0.58,4.58,1.02,13.25,0.53,1.8404,2
882,264,10.3,475,7.49,0.74,0.88,10.56,0.45,7.36,1.87,10.63,0.63,1.5136,0
883,220,8.6,444,7.43,0.65,0.72,11.70,0.37,0.66,0.90,2.19,1.82,1.2384,0


In [421]:
data2 = pd.read_csv('../Data/Dataset2.csv')
data2

,zcta,time_period,population,Start date,end date,case count,test count,positive tests,case rate,test rate,positivity rate
0,95129,32,39741,10/11/2020,10/31/2020,22.0,2543.0,23.0,2.6,304.7,0.9
1,95129,43,39741,5/30/2021,6/19/2021,NaN,3315.0,14.0,1.1,397.2,0.4
2,95129,40,39741,3/28/2021,4/17/2021,34.0,4816.0,37.0,4.1,577.1,0.8
3,95129,55,39741,2/6/2022,2/26/2022,110.0,10194.0,175.0,13.2,1221.5,1.7
4,95129,44,39741,6/20/2021,7/10/2021,14.0,3033.0,17.0,1.7,363.4,0.6
...,...,...,...,...,...,...,...,...,...,...,...
332,94085,59,23223,1-May,21-May,165.0,2315.0,192.0,33.8,474.7,8.3
333,94085,63,23223,24-Jul,13-Aug,150.0,1348.0,190.0,30.8,276.4,14.1
334,94085,61,23223,12-Jun,2-Jul,219.0,1696.0,255.0,44.9,347.8,15.0
335,94085,27,23223,28-Jun,18-Jul,53.0,1379.0,61.0,10.9,282.8,4.4


# Analyse des caractéristiques des attributs du dataset

<h3 style="color:#FFFFFF;">Calculer les mesures de tendance centrale et en déduire les symétries.</h3>

In [422]:
class Dataset:
    def __init__(self, data):
        
        self.data = data
        self.data = self.data.dropna()
        self.data = self.data.apply(pd.to_numeric, errors='coerce')
        self.data = self.data.dropna()

    def describe(self):
        return self.data.describe()

    def mean(self):
        return self.data.mean(numeric_only=True)

    def median(self):
        return self.data.median(numeric_only=True)

    def mode(self):
        mode = self.data.mode(numeric_only=True)
        return mode.mean()
    
    def tendance(self):
        df = ((self.data.mean(numeric_only=True) - self.data.median(numeric_only=True)) / self.data.mean(numeric_only=True))*100
        # Define conditions and replacement values
        conditions = {'Negative Asymmetric': lambda x: x < -5,
                    'Symmetric': lambda x: -5 <= x <= 5,
                    'Positive Asymmetric': lambda x: x > 5}

        # Apply conditions using apply and a lambda function
        df = df.apply(lambda col: np.select([(condition(col)) for condition in conditions.values()], list(conditions.keys())))

        return df
    

Set = Dataset(data1)
Set.describe()  
Set.tendance()

N                      Symmetric
P            Positive Asymmetric
K            Positive Asymmetric
pH                     Symmetric
EC                     Symmetric
OC                     Symmetric
S            Positive Asymmetric
Zn           Positive Asymmetric
Fe           Positive Asymmetric
Cu                     Symmetric
Mn                     Symmetric
B            Positive Asymmetric
OM                     Symmetric
Fertility    Negative Asymmetric
dtype: object

<h3 style="color:#FFFFFF;">Construire une boîte à moustache et afficher les données aberrantes.</h3>


In [423]:
"""boxes = []
for i, column in enumerate(Set.data.columns, 1):
    plt.figure(figsize=(6, 4))
    box = Set.data.boxplot(column=column,return_type='dict')
    boxes.append((column,box))
    plt.title(f'Boxplot of {column}')
    print(boxes[i-1][1]['fliers'][0].get_ydata())
    Set.data.hist(column=column,)
    plt.title(f'Histogram of {column}')
    plt.show()"""
    


"boxes = []\nfor i, column in enumerate(Set.data.columns, 1):\n    plt.figure(figsize=(6, 4))\n    box = Set.data.boxplot(column=column,return_type='dict')\n    boxes.append((column,box))\n    plt.title(f'Boxplot of {column}')\n    print(boxes[i-1][1]['fliers'][0].get_ydata())\n    Set.data.hist(column=column,)\n    plt.title(f'Histogram of {column}')\n    plt.show()"

<h3> Suppression des valeurs aberante <h3>

In [424]:
"""for i in range(len(boxes)):
    print(boxes[i][0])
    print(boxes[i][1]['fliers'][0].get_ydata())
    for outlier in boxes[i][1]['fliers'][0].get_ydata():
        data1 = data1[ data1[boxes[i][0]] != outlier] """

"for i in range(len(boxes)):\n    print(boxes[i][0])\n    print(boxes[i][1]['fliers'][0].get_ydata())\n    for outlier in boxes[i][1]['fliers'][0].get_ydata():\n        data1 = data1[ data1[boxes[i][0]] != outlier] "

<h3> Graph de données </h3>

In [425]:
#scatter_matrix(Set.data, figsize=(24, 24), diagonal='hist')

In [426]:
"""for i in data1.columns:
    for j in data1.columns[data1.columns.get_loc(i)+1:]:
        plt.figure(figsize=(6, 4))
        plt.scatter(data1[i], data1[j])
        plt.xlabel(i)
        plt.ylabel(j)
        plt.title(f'Scatter plot of {i} and {j}')
        plt.show()"""

"for i in data1.columns:\n    for j in data1.columns[data1.columns.get_loc(i)+1:]:\n        plt.figure(figsize=(6, 4))\n        plt.scatter(data1[i], data1[j])\n        plt.xlabel(i)\n        plt.ylabel(j)\n        plt.title(f'Scatter plot of {i} and {j}')\n        plt.show()"

# PARTIE RULES

In [427]:
import itertools

data3 = pd.read_csv('../Data/Dataset3.csv')

data3

,Temperature,Humidity,Rainfall,Soil,Crop,Fertilizer
0,"24,87","82,84","295,61",Clayey,rice,DAP
1,"28,69","96,65","178,96",laterite,Coconut,Good NPK
2,"20,27","81,64","270,44",silty clay,rice,MOP
3,"25,07","95,02","192,9",sandy,Coconut,Urea
4,"25,04","95,9","174,8",coastal,Coconut,Urea
...,...,...,...,...,...,...
290,"26,07","96,7","143,53",sandy,Coconut,MOP
291,"25,63","83,53","209,9",silty clay,rice,MOP
292,"24,52","83,54","230,45",Clayey,rice,MOP
293,"25,66","83,47","217,38",Clayey,rice,MOP


In [428]:
list_data3 = data3.values.tolist()
list_data3

[['24,87', '82,84', '295,61', 'Clayey', 'rice', 'DAP'],
 ['28,69', '96,65', '178,96', 'laterite', 'Coconut', 'Good NPK'],
 ['20,27', '81,64', '270,44', 'silty clay', 'rice', 'MOP'],
 ['25,07', '95,02', '192,9', 'sandy', 'Coconut', 'Urea'],
 ['25,04', '95,9', '174,8', 'coastal', 'Coconut', 'Urea'],
 ['20,82', '84,13', '230,22', 'clay loam', 'rice', 'Urea'],
 ['25,95', '93,41', '172,05', 'alluvial', 'Coconut', 'Urea'],
 ['26,49', '80,16', '242,86', 'Clayey', 'rice', 'DAP'],
 ['25,01', '95,59', '165,81', 'coastal', 'Coconut', 'Urea'],
 ['21,87', '80,19', '224,56', 'silty clay', 'rice', 'Urea'],
 ['28,96', '95,16', '222,8', 'alluvial', 'Coconut', 'DAP'],
 ['26,3', '99,66', '215,92', 'laterite', 'Coconut', 'DAP'],
 ['27,51', '94,19', '156,67', 'sandy', 'Coconut', 'Good NPK'],
 ['23,68', '81,04', '233,7', 'clay loam', 'rice', 'DAP'],
 ['25,52', '94,38', '178,73', 'sandy', 'Coconut', 'Urea'],
 ['25,79', '82,11', '243,51', 'Clayey', 'rice', 'Urea'],
 ['24,28', '80,3', '231,09', 'silty clay', '

In [429]:
def transaction_to_item(transaction_data):
    # Crée un dictionnaire pour compter le nombre d'occurrences de chaque élément
    dict_item = {}

    for transaction in transaction_data:
        for item in transaction:
            if item in dict_item:
                dict_item[item] += 1
            else:
                dict_item[item] = 1
    
    dict_item = {cle: dict_item[cle] for cle in sorted(dict_item , key=str.lower)}
    return dict_item

def filtre_dictionnaire(dictionnaire, Min_Supp):
    # Crée un nouveau dictionnaire pour stocker les éléments filtrés
    dictionnaire_filtré = {}

    for clé, valeur in dictionnaire.items():
        if valeur >= Min_Supp:
            dictionnaire_filtré[clé] = valeur

    return dictionnaire_filtré

def tuple_to_item(tuples):

    tuplee = list(tuples.keys())[0]

    if (type(tuplee) == tuple):
        tuples = list(tuples.keys())
        items = []
        for one_tuple in tuples:
            for item in one_tuple:

                if item not in items:
                    items.append(item)  
    else:
        items = []
        for item in tuples:
            if item not in items:
                items.append(item)  

    return items

def items_to_tuple(Items, Taille_tuple):
    Items.sort(key=str.lower)
    all_tuples = list(itertools.combinations(Items, Taille_tuple))

    dictionnaire = {}
    for one_tuple in all_tuples:
        dictionnaire[one_tuple] = 0

    return dictionnaire

def check_if_line_contains_elements(data, elements):
    for i in elements:
        count = data.count(i)
        if count == 0:
            return False
    return True

    
def update_frequence(transaction_data, tuples):
    Lk = tuples
    for ligne in transaction_data:
        for (item, valeur) in Lk.items():
            if check_if_line_contains_elements(ligne,item):
                Lk[item] += 1
    return Lk

In [430]:
def Apriori(transaction_data, Min_Supp_number = -1, Min_Supp_percent = -1):
    
    if (Min_Supp_number == -1 and Min_Supp_percent == -1):
        print("Erreur: Veuillez entrer un seuil")
        return -1
    
    if (Min_Supp_number != -1):
        Min_Supp = Min_Supp_number

    if (Min_Supp_percent != -1):
        Min_Supp = len(transaction_data)*Min_Supp_percent/100


    phase = 1
    Lk = {}
    Ck = {}
    Ck[1] = {}
    Lk[1] = {}

    # Phase 1
    Ck[phase] =     transaction_to_item(transaction_data)
    print ("Ck["    ,phase, "]\t", Ck[phase])
    Lk[phase] =     filtre_dictionnaire(Ck[phase], Min_Supp)
    print ("Lk["    ,phase, "]\t", Lk[phase])

    
    #debut boucle while

    while (Lk[phase] != {}):

        phase += 1

        #Générer toutes les combinaisons possibles formant des k-itemsets

        items       =   tuple_to_item (Lk[phase-1])
        Ck[phase]   =   items_to_tuple(items, phase)
        Ck[phase] =     update_frequence    (transaction_data, Ck[phase])

        if (len(Ck[phase]) == 0):
            break

        print ("Ck["    ,phase, "]\t", Ck[phase])

        temporary =     filtre_dictionnaire (Ck[phase], Min_Supp)
        
        if (len(temporary) == 0):
            print ("Fin de apriori")
            break

        Lk[phase] = temporary
        print ("Lk["    ,phase, "]\t", Lk[phase])
    return Lk

Min_Supp = 50


result_dict = Apriori(list_data3,Min_Supp_number=  Min_Supp)
print(result_dict)


Ck[ 1 ]	 {'131,09': 1, '131,21': 1, '131,25': 2, '131,34': 1, '132,15': 2, '134,13': 2, '134,84': 3, '135,13': 1, '135,42': 1, '141,32': 2, '142,14': 2, '142,94': 1, '143,53': 2, '144,16': 2, '144,33': 2, '144,63': 1, '145,06': 2, '145,42': 1, '145,54': 1, '145,93': 2, '147,17': 2, '147,89': 1, '148,3': 2, '148,84': 1, '149,22': 1, '149,24': 2, '150,05': 1, '150,2': 1, '150,95': 1, '151,08': 2, '152,42': 1, '156,67': 1, '156,77': 2, '157,24': 1, '157,76': 2, '157,92': 1, '160,39': 2, '160,73': 2, '162,84': 1, '164,92': 1, '165,81': 2, '166,57': 1, '167,05': 2, '171,63': 1, '171,65': 2, '171,76': 2, '171,95': 1, '172,05': 2, '172,48': 2, '174,63': 1, '174,8': 2, '178,73': 2, '178,81': 2, '178,96': 1, '179,82': 2, '181,69': 1, '182,45': 2, '182,56': 2, '182,65': 2, '182,9': 1, '183,62': 2, '183,68': 1, '184,76': 1, '185,28': 2, '185,5': 1, '185,95': 2, '186,75': 2, '188,41': 2, '188,93': 1, '189,67': 2, '191,01': 2, '191,3': 1, '191,77': 1, '191,95': 2, '192,32': 1, '192,77': 1, '192,9':

In [431]:
result_dict

{1: {'Clayey': 54,
  'coastal': 50,
  'Coconut': 152,
  'DAP': 86,
  'MOP': 75,
  'rice': 143,
  'silty clay': 65,
  'Urea': 101},
 2: {('Clayey', 'rice'): 54,
  ('coastal', 'Coconut'): 50,
  ('Coconut', 'Urea'): 53,
  ('rice', 'silty clay'): 65}}

In [432]:
k_items = []

for cle in range (1, len(result_dict)+1):
    for item in result_dict[cle]:
        if item not in k_items:
            k_items.append(item)


print (k_items)


['Clayey', 'coastal', 'Coconut', 'DAP', 'MOP', 'rice', 'silty clay', 'Urea', ('Clayey', 'rice'), ('coastal', 'Coconut'), ('Coconut', 'Urea'), ('rice', 'silty clay')]


In [433]:
def calcul_taille_tuple(one_tuple):
    
    if not isinstance(one_tuple, (list, tuple  )):
        return 1
    return len(one_tuple)


def tuple_to_items(tuples):
    items = []
    if calcul_taille_tuple(tuples) == 1:
        items.append(tuples)
    
    else:
        for i in tuples:
            items.append(i)  

    return items


def ont_elements_en_commun(liste1, liste2):
    for element in liste1:
        if element in liste2:
            return True
    return False


In [434]:
def generate_association_rules(k_items):
    rules = []

    for antecedent in k_items:
        for consequent in k_items:
            # Vérifier si l'antécédent et la conséquence sont différents
            if antecedent != consequent and not ont_elements_en_commun(tuple_to_items(antecedent), tuple_to_items(consequent) ):
                # Vérifier si l'antécédent et la conséquence ne partagent aucun item ou partie d'item
                # Ajouter la règle si toutes les conditions sont remplies
                rules.append((antecedent, consequent))

    # Afficher les règles générées
    for rule in rules:
        print(rule[0], '->', rule[1])

    return rules

rules = generate_association_rules(k_items)


Clayey -> coastal
Clayey -> Coconut
Clayey -> DAP
Clayey -> MOP
Clayey -> rice
Clayey -> silty clay
Clayey -> Urea
Clayey -> ('coastal', 'Coconut')
Clayey -> ('Coconut', 'Urea')
Clayey -> ('rice', 'silty clay')
coastal -> Clayey
coastal -> Coconut
coastal -> DAP
coastal -> MOP
coastal -> rice
coastal -> silty clay
coastal -> Urea
coastal -> ('Clayey', 'rice')
coastal -> ('Coconut', 'Urea')
coastal -> ('rice', 'silty clay')
Coconut -> Clayey
Coconut -> coastal
Coconut -> DAP
Coconut -> MOP
Coconut -> rice
Coconut -> silty clay
Coconut -> Urea
Coconut -> ('Clayey', 'rice')
Coconut -> ('rice', 'silty clay')
DAP -> Clayey
DAP -> coastal
DAP -> Coconut
DAP -> MOP
DAP -> rice
DAP -> silty clay
DAP -> Urea
DAP -> ('Clayey', 'rice')
DAP -> ('coastal', 'Coconut')
DAP -> ('Coconut', 'Urea')
DAP -> ('rice', 'silty clay')
MOP -> Clayey
MOP -> coastal
MOP -> Coconut
MOP -> DAP
MOP -> rice
MOP -> silty clay
MOP -> Urea
MOP -> ('Clayey', 'rice')
MOP -> ('coastal', 'Coconut')
MOP -> ('Coconut', 'Urea'

In [435]:
import math


def union_items_tuples(tuple1, tuple2):
    items = []
    
    if calcul_taille_tuple(tuple1) == 1:
        items.append(tuple1)
    else:
        for i in tuple1:
            items.append(i)
    
    if calcul_taille_tuple(tuple2) == 1:
        items.append(tuple2)
    else:
        for i in tuple2:
            items.append(i)
    
    items.sort(key=str.lower)
    return items


def calculate_metrics(support_AB, support_A, support_B, total_transactions, size_A, size_B):

    confidence = support_AB / support_A if support_A != 0 else 0

    lift_denominator = (support_A / total_transactions) * (support_B / total_transactions)
    lift = support_AB / lift_denominator if lift_denominator != 0 else 0

    cosine_numerator = support_AB
    cosine_denominator = math.sqrt(support_A * support_B) * math.sqrt(size_A * size_B)
    cosine_similarity = cosine_numerator / cosine_denominator if cosine_denominator != 0 else 0

    return confidence, lift, cosine_similarity

In [436]:


def calcul_confiance(rules, result_dict):

    confiance = {}
    lift = {}
    cosine = {}

    for rule in rules:
        
        A = rule[0]
        B = rule[1]
        A_u_B = union_items_tuples(A, B)
        
        if calcul_taille_tuple(A_u_B) <= len(result_dict):

            
            temp = result_dict[calcul_taille_tuple(A_u_B)]
            # Convert A_u_B to a tuple before using it as a key
            A_u_B_tuple = tuple(A_u_B)


            if A_u_B_tuple in temp.keys():
                Supp_A = result_dict[calcul_taille_tuple(A)][A]
                Supp_A_u_B = result_dict[calcul_taille_tuple(A_u_B_tuple)][A_u_B_tuple]

        
                confiance[A, B],lift[A, B], cosine[A, B]  = calculate_metrics(Supp_A_u_B, Supp_A, Supp_A, len(list_data3), calcul_taille_tuple(A), calcul_taille_tuple(B))


                


    return confiance, lift, cosine
        
confiance, lift, cosine = calcul_confiance(rules, result_dict)




In [437]:
print ("confiance ")
for i in confiance:
    print (i[0],'->',i[1],':', confiance[i])

confiance 
Clayey -> rice : 1.0
coastal -> Coconut : 1.0
Coconut -> coastal : 0.32894736842105265
Coconut -> Urea : 0.34868421052631576
rice -> Clayey : 0.3776223776223776
rice -> silty clay : 0.45454545454545453
silty clay -> rice : 1.0
Urea -> Coconut : 0.5247524752475248


In [438]:
print ("lift ")
for i in lift:
    print (i[0],'->',i[1],':', lift[i])
    

lift 
Clayey -> rice : 1611.5740740740741
coastal -> Coconut : 1740.5000000000002
Coconut -> coastal : 188.33318905817174
Coconut -> Urea : 199.63318040166206
rice -> Clayey : 229.80830358452735
rice -> silty clay : 276.6211061665607
silty clay -> rice : 1338.8461538461538
Urea -> Coconut : 452.14439760807767


In [439]:
print ("cosine ")
for i in cosine:
    print (i[0],'->',i[1],':', cosine[i])

cosine 
Clayey -> rice : 1.0
coastal -> Coconut : 1.0
Coconut -> coastal : 0.32894736842105265
Coconut -> Urea : 0.34868421052631576
rice -> Clayey : 0.3776223776223776
rice -> silty clay : 0.45454545454545453
silty clay -> rice : 1.0
Urea -> Coconut : 0.5247524752475248


In [440]:
Min_Conf = 0.3

def afficher_regles_confiance(confiance, Min_Conf):
    for i in confiance:
        if confiance[i] >= Min_Conf:
            print (i[0],'->',i[1],':', confiance[i])

            
afficher_regles_confiance(confiance, Min_Conf)

Clayey -> rice : 1.0
coastal -> Coconut : 1.0
Coconut -> coastal : 0.32894736842105265
Coconut -> Urea : 0.34868421052631576
rice -> Clayey : 0.3776223776223776
rice -> silty clay : 0.45454545454545453
silty clay -> rice : 1.0
Urea -> Coconut : 0.5247524752475248
